# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [3]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [4]:
#just double checking things...
print df.prestige.value_counts()
print df.admit.value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64
0    271
1    126
Name: admit, dtype: int64


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [5]:
dummy_prestige = pd.get_dummies(df.prestige, prefix="pre")
dummy_prestige.head(2)

,pre_1.0,pre_2.0,pre_3.0,pre_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0


In [6]:
dummy_prestige.rename(columns={'pre_1.0': 'pre_1','pre_2.0': 'pre_2','pre_3.0': 'pre_3','pre_4.0': 'pre_4'}, inplace = True)

dummy_prestige.head(2)

,pre_1,pre_2,pre_3,pre_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0


In [7]:
df = df.join([dummy_prestige])
df.head(2)

,admit,gre,gpa,prestige,pre_1,pre_2,pre_3,pre_4
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: In this case we need 3 since the 4th can be inferred by knowing the other 3. Though technically don't we only need 1? Then run 4 models trying out each prestige variable on its own?

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [8]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_prestige.ix[:, 'pre_1':])
print handCalc.head()

   admit    gre   gpa  pre_1  pre_2  pre_3  pre_4
0      0  380.0  3.61    0.0    0.0    1.0    0.0
1      1  660.0  3.67    0.0    0.0    1.0    0.0
2      1  800.0  4.00    1.0    0.0    0.0    0.0
3      1  640.0  3.19    0.0    0.0    0.0    1.0
4      0  520.0  2.93    0.0    0.0    0.0    1.0


In [9]:
# crosstab prestige 1 admission
pd.crosstab(df.admit, df.pre_1, margins=True)

pre_1,0.0,1.0,All
admit,,,
0,243,28,271
1,93,33,126
All,336,61,397


In [10]:
# frequency table cutting prestige and whether or not someone was admitted

# changes absolute numbers to percentages
def percConvert(ser):
  return ser/float(ser[-1])
pd.crosstab(df["pre_1"],df["admit"],margins=True).apply(percConvert, axis=1)

admit,0,1,All
pre_1,,,
0.0,0.723214,0.276786,1.0
1.0,0.459016,0.540984,1.0
All,0.682620,0.317380,1.0


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [11]:
# probability: 61 attended and 33 were admitted:
prob_1 = 33./61
print "The probability is {}".format(prob_1)
# odds: 33 admitted to 28 not admitted
odds_1 = 33./28
print "But the odds are {}".format(odds_1)

The probability is 0.540983606557
But the odds are 1.17857142857


#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [12]:
# probability: 336 didn't attend and 93 were admitted:
prob_not1 = 93./336
print "The probability is {}".format(prob_not1)
# odds: 93 admitted to 243 not admitted
odds_not1 = 93./243
print "But the odds are {}".format(odds_not1)

The probability is 0.276785714286
But the odds are 0.382716049383


#### Question 3.3. Calculate the odds ratio.

In [13]:
odds_1/odds_not1

3.079493087557604

#### Question 3.4. Write this finding in a sentenance:

Answer: The odds of being admitted are ~3 times higher (or 3 to 1 odds) if you attend a no 1 ranked college as compared to a lower ranked college 

#### Question 3.5. Print the cross tab for prestige_4.

In [14]:
pd.crosstab(df.admit, df.pre_4, margins=True)

pre_4,0.0,1.0,All
admit,,,
0,216,55,271
1,114,12,126
All,330,67,397


#### Question 3.6. Calculate the OR.

In [15]:
#odds admitted if attended #4 college
#12 admitted to 55 not admitted
odds_4 = 12.0/55
print "The odds of admittance are {} if you attended a #4 ranked college".format(odds_4)
#odds admitted if did not attend #4 college
#114 admitted to 216 not admitted
odds_no4 = 114.0/216
print "The odds of admittance are {} if you didn't attend a #4 ranked college".format(odds_no4)

odds_4/odds_no4

The odds of admittance are 0.218181818182 if you attended a #4 ranked college
The odds of admittance are 0.527777777778 if you didn't attend a #4 ranked college


0.4133971291866028

#### Question 3.7. Write this finding in a sentence.

In [16]:
1/(odds_4/odds_no4)

2.418981481481482

Answer: The odds of being admitted are ~2.4(reciprocal of OR) times less if you attend a no 4 ranked college as compared to a higher ranked college

## Part 4. Analysis

In [17]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_prestige.ix[:, 'pre_2':])
print data.head()

   admit    gre   gpa  pre_2  pre_3  pre_4
0      0  380.0  3.61    0.0    1.0    0.0
1      1  660.0  3.67    0.0    1.0    0.0
2      1  800.0  4.00    0.0    0.0    0.0
3      1  640.0  3.19    0.0    0.0    1.0
4      0  520.0  2.93    0.0    0.0    1.0


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [18]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [19]:
train_cols = data[['gre', 'gpa', 'pre_2', 'pre_3', 'pre_4']]

#### Question 4.2. Fit the model.

In [20]:
##Oh, oops this is logistic regression with sklearn
model = linear_model.LogisticRegression()
model.fit(train_cols)

NameError: name 'linear_model' is not defined

In [21]:
#with statsmodels
logit = sm.Logit(data['admit'], train_cols)
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


#### Question 4.3. Print the summary results.

In [22]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Thu, 14 Jul 2016   Pseudo R-squ.:                 0.05722
Time:                        20:38:41   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.308      0.191        -0.001     0.003
gpa           -0.1323      0.195     -0.680      0.497        -0.514     0.249
pre_2         -0.9562      0.302     -3.171      0.002        -1.547    -0.365
pre_3         -1.5375      0.332     -4.627      0.000        -2.189    -0.886
pre_4         -1.8699      0.401     -4.658      0.000        -2.657    -1.083
==============================================================================
"""

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [23]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

           2.5%     97.5%        OR
gre    0.999320  1.003420  1.001368
gpa    0.598303  1.282800  0.876073
pre_2  0.212826  0.694082  0.384342
pre_3  0.112055  0.412207  0.214918
pre_4  0.070176  0.338540  0.154135


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: The odds of being admitted are ~2.6 times less (reciprocal of OR) if you attend a no 4 ranked college as compared to a no 1 ranked college 

In [24]:
1/0.384342

2.601849394549646

#### Question 4.6. Interpret the OR of GPA.

Answer: The odds are being admitted with a perfect GPA are 1.14 to 1 (or 87.6%)

In [25]:
1/0.876073

1.1414573899663614

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [26]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [27]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [28]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'admit']
dummy_prestige = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_prestige.columns = ['pre_1', 'pre_2', 'pre_3', 'pre_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_prestige.ix[:, 'pre_2':])


KeyError: "['intercept'] not in index"

#### Question 5.2. Make predictions on the enumerated dataset.

!! Very very confused at this point... !!

#### Question 5.3. Interpret findings for the last 4 observations.

Answer:

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.